In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!pip install simpletransformers

In [ ]:
import pandas as pd
from simpletransformers.model import TransformerModel
import traceback
import os

In [ ]:
os.listdir('../input/bert-model/')

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
df = pd.read_csv('../input/tweets/dataset.tsv',sep='\t',header=None)

In [ ]:
args = {
   'output_dir': 'output/',
   'cache_dir': 'cache/',
   'fp16': False,
   'fp16_opt_level': 'O1',
   'max_seq_length': 128,
   'train_batch_size': 8,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 1,
   'weight_decay': 0,
   'learning_rate': 3e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,'overwrite_output_dir': False,
   'reprocess_input_data': False,
#     'num_labels' : 4
}

In [ ]:
data = []
for i,val in df.iterrows():
    try:
        data.append(list([val[2],int(val[1])]))
    except:
        pass

In [ ]:
model = TransformerModel('xlnet','xlnet-base-cased',args=args,use_cuda=False,num_labels=4)

In [ ]:
train_df = pd.DataFrame(data)
try:
    model.train_model(train_df)
except:
    traceback.print_exc()

In [ ]:
import pickle
with open('../input/tweets/model.pkl','wb') as f:
    pickle.dump(model,f)

In [ ]:
with open('output/model.pkl','rb') as f:
    model = pickle.load(f)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

test = pd.read_csv('../input/tweets-testing/dataset_test.tsv',sep='\t',header = None)
test_data = []
test_labels = []
for x in test[2]:
    test_data.append(x)

for x in test[1]:
    test_labels.append(x)
    
test_predictions = model.predict(test_data)

print("accuracy = ",accuracy_score(test_labels, test_predictions[0]))
print("f1Score = ",f1_score(test_labels, test_predictions[0],average='macro'))
print("matrix = ",confusion_matrix(test_labels, test_predictions[0]))